In [6]:
# train_gru.py

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
import re
import html
from tqdm import tqdm

# ----------------------------
# 1️⃣ Load & clean dataset
# ----------------------------
df = pd.read_csv(r"R:\AIniverse\PROJECTS\mainstream\senti\data\train.csv\train.csv")  # path to your CSV

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

def clean_text(text):
    text = str(text)
    text = html.unescape(text)
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"\S+@\S+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df['comment_text'] = df['comment_text'].apply(clean_text)

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# ----------------------------
# 2️⃣ Build vocabulary
# ----------------------------
from collections import Counter
from itertools import chain

all_text = list(train_df['comment_text'])
counter = Counter(chain.from_iterable([x.split() for x in all_text]))
vocab = {w:i+2 for i,(w,c) in enumerate(counter.most_common())}  # +2 for PAD=0, UNK=1
vocab['<PAD>'] = 0
vocab['<UNK>'] = 1
vocab_size = len(vocab)

def encode_text(text, max_len=128):
    tokens = [vocab.get(w,1) for w in text.split()]
    if len(tokens) < max_len:
        tokens += [0]*(max_len-len(tokens))
    else:
        tokens = tokens[:max_len]
    return tokens
# Build vocabulary from training comments
from collections import Counter
import torch

all_text = " ".join(train_df['comment_text'].tolist())
words = all_text.split()
word_counts = Counter(words)

# Create vocab: word -> index (0=padding, 1=unknown)
vocab = {"<PAD>":0, "<UNK>":1}
for i, word in enumerate(word_counts.keys(), start=2):
    vocab[word] = i

# Save vocab
torch.save(vocab, "gru_vocab.pt")  # or bilstm_vocab.pt / transformer_vocab.pt


In [1]:

# ----------------------------
# 3️⃣ Dataset class
# ----------------------------
class ToxicDataset(Dataset):
    def __init__(self, df):
        self.texts = [encode_text(x) for x in df['comment_text']]
        self.labels = torch.tensor(df[label_cols].values, dtype=torch.float32)
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return {'input_ids': torch.tensor(self.texts[idx], dtype=torch.long),
                'labels': self.labels[idx]}

train_dataset = ToxicDataset(train_df)
val_dataset = ToxicDataset(val_df)
test_dataset = ToxicDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [2]:

# ----------------------------
# 4️⃣ GRU + Attention Model
# ----------------------------
class AttentionGRU(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=128, output_dim=6):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.gru = nn.GRU(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.attn = nn.Linear(hidden_dim*2, 1)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim*2, output_dim)

    def forward(self, input_ids):
        x = self.embedding(input_ids)
        gru_out, _ = self.gru(x)
        attn_weights = torch.softmax(self.attn(gru_out).squeeze(-1), dim=1).unsqueeze(-1)
        weighted = gru_out * attn_weights
        pooled = weighted.sum(1)
        out = self.dropout(pooled)
        return self.fc(out)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AttentionGRU(vocab_size).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-3)

# ----------------------------
# 5️⃣ Training with Early Stopping
# ----------------------------
epochs = 20
patience = 3
best_val_loss = float('inf')
early_stop_counter = 0

for epoch in range(epochs):
    model.train()
    train_loss = 0
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    avg_train_loss = train_loss/len(train_loader)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    avg_val_loss = val_loss/len(val_loader)
    print(f"Epoch {epoch+1}: Train Loss {avg_train_loss:.4f}, Val Loss {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), "gru_toxic_model.pt")
        print("✅ Model improved. Saved.")
    else:
        early_stop_counter +=1
        if early_stop_counter>=patience:
            print("⛔ Early stopping triggered.")
            break

100%|██████████| 3990/3990 [01:14<00:00, 53.49it/s, loss=0.0019] 


Epoch 1: Train Loss 0.0614, Val Loss 0.0494
✅ Model improved. Saved.


100%|██████████| 3990/3990 [01:13<00:00, 53.98it/s, loss=0.1]    


Epoch 2: Train Loss 0.0425, Val Loss 0.0509


100%|██████████| 3990/3990 [01:13<00:00, 54.09it/s, loss=0.00262] 


Epoch 3: Train Loss 0.0341, Val Loss 0.0497


100%|██████████| 3990/3990 [01:13<00:00, 54.07it/s, loss=0.00633] 


Epoch 4: Train Loss 0.0289, Val Loss 0.0556
⛔ Early stopping triggered.


In [3]:

# ----------------------------
# 6️⃣ Evaluation
# ----------------------------
model.load_state_dict(torch.load("gru_toxic_model.pt"))
model.eval()
all_labels = []
all_preds = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].cpu().numpy()
        outputs = torch.sigmoid(model(batch['input_ids'].to(device))).cpu().numpy()
        preds = (outputs>0.5).astype(int)
        all_labels.append(labels)
        all_preds.append(preds)

import numpy as np
all_labels = np.vstack(all_labels)
all_preds = np.vstack(all_preds)

for i,col in enumerate(label_cols):
    print(f"--- {col} ---")
    print("Accuracy: ", (all_labels[:,i]==all_preds[:,i]).mean())
    print("F1-score:", f1_score(all_labels[:,i], all_preds[:,i]))
    print("ROC-AUC:", roc_auc_score(all_labels[:,i], all_preds[:,i]))


C:\Users\anmol\AppData\Local\Temp\ipykernel_9544\661818755.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("gru_toxic_model.pt"))


--- toxic ---
Accuracy:  0.9621506454442913
F1-score: 0.789839944328462
ROC-AUC: 0.865771110228126
--- severe_toxic ---
Accuracy:  0.9897230229352049
F1-score: 0.3643410852713178
ROC-AUC: 0.6435107028189665
--- obscene ---
Accuracy:  0.9791953878932197
F1-score: 0.7950617283950617
ROC-AUC: 0.8721952405882986
--- threat ---
Accuracy:  0.9977440782052889
F1-score: 0.18181818181818182
ROC-AUC: 0.5539598388665659
--- insult ---
Accuracy:  0.9733049254292517
F1-score: 0.7283163265306123
ROC-AUC: 0.8471039603960395
--- identity_hate ---
Accuracy:  0.9922922672014037
F1-score: 0.3128491620111732
ROC-AUC: 0.6010384030487916


In [ ]:
import torch
import torch.nn.functional as F

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

def predict_toxicity(comment, model, vocab, device, threshold=0.5):
    # 1️⃣ Clean comment
    comment_clean = clean_text(comment)

    # 2️⃣ Encode using vocab
    tokens = [vocab.get(w,1) for w in comment_clean.split()]
    if len(tokens) < 128:
        tokens += [0]*(128 - len(tokens))
    else:
        tokens = tokens[:128]
    input_ids = torch.tensor(tokens).unsqueeze(0).to(device)

    # 3️⃣ Model prediction
    model.eval()
    with torch.no_grad():
        logits = model(input_ids)
        probs = torch.sigmoid(logits).squeeze().cpu().numpy()

    # 4️⃣ Prepare clean output
    result = {}
    for i, label in enumerate(label_cols):
        pred = int(probs[i] >= threshold)
        result[label] = {"prob": float(probs[i]), "pred": pred, "label": "Yes" if pred else "No"}

    # 5️⃣ Print neatly
    print(f"\nComment: {comment_clean}\n")
    for label in label_cols:
        print(f"{label:15}: {result[label]['label']:3} (prob: {result[label]['prob']:.2f})")

    return result

# ✅ Example usage
comment = "you piece of shit!"
predict_toxicity(comment, model, vocab, device)

In [22]:
for w in ["you", "piece", "of", "shit"]:
    print(w, "->", vocab.get(w, "<UNK>"))


you -> 22
piece -> 1059
of -> 28
shit -> 2557
